In [ ]:
import sys
sys.path.append('..')

In [ ]:
import matplotlib.pyplot as plt
import torch

from model_manager import HingeLoss

out_dir = "../output_random"
n_layers, graph_out_dim = 10, 10
epoch_start, epoch_end = 1, 8
epochs = [i for i in range(epoch_start, epoch_end + 1)]

train_losses = []
eval_losses, eval_preds, eval_targets = [], [], []
train_losses, train_preds, train_targets = [], [], []

criterion = HingeLoss()

for epoch in epochs:
    epoch_data = torch.load(f"{out_dir}/n_layers={n_layers}_graph_out_dim={graph_out_dim}_epoch={epoch}.pt")

    epoch_tt, epoch_tp = epoch_data["train_target"], epoch_data["train_preds"] 
    epoch_et, epoch_ep = epoch_data["eval_target"], epoch_data["eval_preds"]

    eval_targets.extend(epoch_et.tolist())
    eval_preds.extend(epoch_ep.tolist())

    train_targets.extend(epoch_tt.tolist())
    train_preds.extend(epoch_tp.tolist())

    train_losses.append(criterion(epoch_tp, epoch_tt).item())
    eval_losses.append(criterion(epoch_ep, epoch_et).item())

eval_size = len(eval_preds) // len(epochs)

In [ ]:
import matplotlib.pyplot as plt

# Assuming epochs, train_losses, and eval_losses are already defined
plt.figure(figsize=(10, 6))

plt.plot(
    epochs,
    train_losses,
    label="Training Loss",
    linestyle="-",
    marker="o",
    color="blue",
    markersize=8,
    linewidth=2,
)
plt.plot(
    epochs,
    eval_losses,
    label="Validation Loss",
    linestyle="--",
    marker="x",
    color="red",
    markersize=8,
    linewidth=2,
)

plt.legend(fontsize=14, loc="upper right")
plt.title("Training and Validation Losses over Epochs", fontsize=18, weight="bold")
plt.xlabel("Epoch", fontsize=16, weight="bold")
plt.ylabel("Hinge Loss", fontsize=16, weight="bold")
plt.grid(True, which="both", linestyle="--", linewidth=0.5)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.gca().spines["top"].set_visible(False)
plt.gca().spines["right"].set_visible(False)
plt.gca().spines["left"].set_linewidth(1.5)
plt.gca().spines["bottom"].set_linewidth(1.5)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming epochs, eval_preds, and eval_size are already defined
gr = 20
dist_graph_n_epochs = 3
max_y = 1500
label_size=18
title_size = 20
ax_size = 18

# Create subplots in a single row
fig, axes = plt.subplots(1, dist_graph_n_epochs, figsize=(20, 6), sharey=True)

for idx in range(dist_graph_n_epochs):
    i = idx * eval_size
    epoch_eval_preds = eval_preds[i : i + eval_size]
    epoch_eval_targets = eval_targets[i : i + eval_size]

    pos_preds = [
        pred
        for pred, target in zip(epoch_eval_preds, epoch_eval_targets)
        if target == 1
    ]
    neg_preds = [
        pred
        for pred, target in zip(epoch_eval_preds, epoch_eval_targets)
        if target == -1
    ]

    axes[idx].hist(
        pos_preds,
        bins=100,
        color="blue",
        range=(-gr, gr),
        alpha=0.5,
        label="Positive Class",
    )
    axes[idx].hist(
        neg_preds,
        bins=100,
        color="green",
        range=(-gr, gr),
        alpha=0.5,
        label="Negative Class",
    )
    axes[idx].set_title(f"Epoch {epochs[idx]}", fontsize=16)
    axes[idx].set_xlim([-gr, gr])
    axes[idx].set_ylim([0, max_y])
    axes[idx].tick_params(axis="x", labelsize=14)
    axes[idx].tick_params(axis="y", labelsize=14)
    if idx == 0:
        axes[idx].legend(fontsize=14)


# Add common labels
fig.text(0.5, 0.04, "Prediction Values", ha="center", fontsize=ax_size)
fig.text(0.04, 0.5, "Frequency", va="center", rotation="vertical", fontsize=ax_size)

# Adjust layout
plt.tight_layout(rect=[0.05, 0.05, 1, 1])
plt.show()